In [1]:
#profitableHospital-project

In [2]:
#dependencies
import pandas as pd
import requests
import json
import scipy.stats as st
from scipy.stats import linregress
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import tensorflow
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import joblib

In [3]:
# ... File Imports

In [4]:
#2017 Hospital Data from Centers for Medicare Services
df = pd.read_csv("Medicare_Hospital_Cost_Report_PUF_2015.csv")
df.head(2)

C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (104,105) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,CCN Facility Type,...,Total Other Income,Total Income,Total Other Expenses,Net Income,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone SCHIP,Stand-Alone SCHIP Charges,Report Record Number
0,454129,GEORGETOWN BEHAVIORAL HEALTH INSTITU,3101 S AUSTIN AVENUE,GEORGETOWN,TX,78626,NaN,NaN,NaN,PH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,556684
1,110050,MURRAY MEDICAL CENTER,707 OLDS ELLIJAY HIGHWAY,CHATSWORTH,GA,30705,MURRAY,19140.0,U,STH,...,517094.0,-1003149.0,NaN,-1003149.0,0.257989,3019425.0,4601624.0,NaN,NaN,566354


In [5]:
#Quick Check
df['Provider CCN'].count()

6257

In [6]:
#Certificate of Need Status by State File
conTable = pd.read_csv("certificateofneedstatesTable.csv")
conTable.head(2)

,State Code,CON
0,AK,1
1,AL,1


In [7]:
#String to Numeric Conversion Table for Urban v. Rural designation
urbanTable = pd.read_csv("ruralversusurbanTable.csv")
urbanTable.head()

,Rural Versus Urban,Urban
0,U,1
1,R,0


In [8]:
#Lattitude & Longitude for the 2017 hospital listing
geoTable = pd.read_csv("ccn2geocode.csv")
geoTable.head(2)

,Provider CCN,lat,lng
0,10001,31.215884,-85.363008
1,10005,34.222103,-86.160908


In [9]:
# ... Appending of source files into one main working file

In [10]:
mainFile1 = df

In [11]:
#Append a lattitude & longitude
mainFile1 = pd.merge(mainFile1, geoTable, how= "left", on=['Provider CCN'])
#mainFile1.head(2)

In [12]:
#Quick Check
mainFile1['Provider CCN'].count()

6257

In [13]:
#Append certificate of need indicator
mainFile1 = pd.merge(mainFile1, conTable, how= "left", on=['State Code'])
#mainFile1.head()

In [14]:
#Quick Check
mainFile1['Provider CCN'].count()

6257

In [15]:
#Append a numeric representation of rural v. urban
mainFile1 = pd.merge(mainFile1, urbanTable, how= "left", on=['Rural Versus Urban'])

In [16]:
#Quick Check
mainFile1['Provider CCN'].count()

6257

In [17]:
mainFile1.head(2)

,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,CCN Facility Type,...,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone SCHIP,Stand-Alone SCHIP Charges,Report Record Number,lat,lng,CON,Urban
0,454129,GEORGETOWN BEHAVIORAL HEALTH INSTITU,3101 S AUSTIN AVENUE,GEORGETOWN,TX,78626,NaN,NaN,NaN,PH,...,NaN,NaN,NaN,NaN,NaN,556684,30.614889,-97.680628,0.0,NaN
1,110050,MURRAY MEDICAL CENTER,707 OLDS ELLIJAY HIGHWAY,CHATSWORTH,GA,30705,MURRAY,19140.0,U,STH,...,0.257989,3019425.0,4601624.0,NaN,NaN,566354,34.779956,-84.783038,1.0,1.0


In [18]:
mainFile1.to_csv("main1_2015.csv", index=False, header=True)

In [19]:
# ... Add calculated fields

In [20]:
mainFile2 = mainFile1

In [21]:
#labor Expense
lbrexpense = mainFile2["Wage-Related Costs (Core)"] + mainFile2["Total Salaries (adjusted)"]
mainFile2["Labor Exp"] = lbrexpense
#wagepercent.head()

In [22]:
#Append calculated field inpatient percent
inpatient = mainFile2["Inpatient Revenue"]/mainFile2["Gross Revenue"]
mainFile2["Inpatient %"] = inpatient
#mainFile2.head()

In [23]:
#Append calculated field outpatient percent
outpatient = mainFile2["Outpatient Revenue"]/mainFile2["Gross Revenue"]
mainFile2["Outpatient %"] = outpatient
#mainFile2.head()

In [24]:
#Append calculated field charity percent
charity = mainFile2["Cost of Charity Care"]/mainFile2["Net Patient Revenue"]
mainFile2["Charity %"] = charity
#mainFile2.head()

In [25]:
#Append calculated field uncompensated percent
uncompensated = mainFile2["Cost of Uncompensated Care"]/mainFile2["Net Patient Revenue"]
mainFile2["Uncompensated %"] = charity
#mainFile2.head()

In [26]:
#Append calculated field revenue per discharge
revdischarge = mainFile2["Net Patient Revenue"]/mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]
mainFile2["Rev/Discharge"] = revdischarge
#mainFile2.head()

In [27]:
#Append calculated field revenue per bed
revbed = mainFile2["Net Patient Revenue"]/mainFile2["Number of Beds"]
mainFile2["Rev Per Bed"] = revbed
#mainFile2.head()

In [28]:
#Append calculated field revenue per bed day
revbed = mainFile2["Net Patient Revenue"]/mainFile2["Total Bed Days Available"]
mainFile2["Rev Per Bed Days"] = revbed
#mainFile2.head()

In [29]:
#Append calculated field revs per fte
revfte = mainFile2["Net Patient Revenue"]/mainFile2["FTE - Employees on Payroll"]
mainFile2["Rev Per FTE"] = revfte
#mainFile2.head()

In [30]:
#Append calculated field discharges per bed
volbed = mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]/mainFile2["Number of Beds"]
mainFile2["Discharges Per Bed"] = volbed
#mainFile2.head()

In [31]:
#Append calculated field discharges per bed day
voldays = mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]/mainFile2["Total Bed Days Available"]
mainFile2["Discharges Per Bed Days"] = voldays
#mainFile2.head()

In [32]:
#Append calculated field discharges per bed
volfte = mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]/mainFile2["FTE - Employees on Payroll"]
mainFile2["Discharge/FTE"] = volbed
#mainFile2.head()

In [33]:
#labor percent
wagepercent = mainFile2["Wage-Related Costs (Core)"] + mainFile2["Total Salaries (adjusted)"]
wagepercent = wagepercent / mainFile2["Net Patient Revenue"]
mainFile2["Labor %"] = wagepercent
#wagepercent.head()

In [34]:
#Overhead percent
overheadpercent = mainFile2["Overhead Non-Salary Costs"] / mainFile2["Net Patient Revenue"]
mainFile2["Overhead %"] = overheadpercent
#overheadpercent.head()

In [35]:
#Depreciation percent
deprpercent = mainFile2["Depreciation Cost"] / mainFile2["Net Patient Revenue"]
mainFile2["Depreciation %"] = deprpercent
#overheadpercent.head()

In [36]:
#labor Per FTE
wagefte = mainFile2["Wage-Related Costs (Core)"] + mainFile2["Total Salaries (adjusted)"]
wagefte = wagefte / mainFile2["FTE - Employees on Payroll"]
mainFile2["LbrExp/FTE"] = wagefte
#wagepercent.head()

In [37]:
#Overhead Expense Per Bed
overbed = mainFile2["Overhead Non-Salary Costs"] / mainFile2["Number of Beds"]
mainFile2["Overhead Exp Per Bed"] = overbed
#wagepercent.head()

In [38]:
#Append calculated field net income percent to file
profit = mainFile2["Net Income"]/mainFile2["Net Patient Revenue"]
mainFile2["Net Income %"] = profit
#mainFile2.head()

In [39]:
# Review net income percent for outlier reduction approach
netinc = mainFile2["Net Income %"]
#netinc.head()

In [40]:
# Box plot and outlier calculations for outlier approach
netinc2 = netinc.dropna(how='any')
chartdata = netinc2
# fig1, ax1 = plt.subplots()
# ax1.set_title('Net Income Percent')
# ax1.set_ylabel('Net Income Percent')
# ax1.boxplot(chartdata)
# plt.show()

In [41]:
#Quartile calculations
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of profit percent is: {lowerq}")
print(f"The upper quartile of profit percent is: {upperq}")
print(f"The interquartile range of profit percent is: {iqr}")
print(f"The the median of profit percent is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile of profit percent is: -0.01906054139096254
The upper quartile of profit percent is: 0.10906152655798192
The interquartile range of profit percent is: 0.12812206794894446
The the median of profit percent is: 0.04292765443003256 
Values below -0.21124364331437923 could be outliers.
Values above 0.3012446284813986 could be outliers.


In [42]:
#Create a bronze, silver, gold score for clasifying net income performance
bins = [-100, -0.030605, 0.113797, 100]
group_names = ["3", "2", "1"]

In [43]:
#Add a bronze, silver, gold scoring system to the dataset
mainFile2["Net Income Score"] = pd.cut(mainFile2["Net Income %"], bins, labels=group_names, include_lowest=True)
#mainFile2.head()

In [44]:
#Quick check on distribution of bronze, silver and gold
mainFile2['Net Income Score'].value_counts()

2    3237
1    1437
3    1320
Name: Net Income Score, dtype: int64

In [45]:
#Pacesetter status
bins2 = [-100, -0.030605, .038045,  0.11379, 100]
group_names2 = ["4th Quartile", "3rd Quartile", "2nd Quartile", "1st Quartile"]

In [46]:
#Add Pacesetter Status to dataframe
mainFile2["Pacesetter Group"] = pd.cut(mainFile2["Net Income %"], bins2, labels=group_names2, include_lowest=True)
#mainFile2.head()

In [47]:
mainFile2['Pacesetter Group'].value_counts()

2nd Quartile    1696
3rd Quartile    1541
1st Quartile    1437
4th Quartile    1320
Name: Pacesetter Group, dtype: int64

In [48]:
#Quick Check
mainFile2['Provider CCN'].count()

6257

In [49]:
#mainFile2.to_csv("main2_2015.csv", index=False, header=True)

In [50]:
# ..... File targeting

In [51]:
mainFile3 = mainFile2
mainFile3['Provider CCN'].count()

6257

In [52]:
#Remove invalid row
mainFile3 = mainFile3.loc[mainFile3['Hospital Name'] != '-7']

In [53]:
#Remove invalid row
mainFile3 = mainFile3.loc[mainFile3['Hospital Name'] != -7]

In [54]:
#Remove invalid row
mainFile3 = mainFile3.loc[mainFile3['Provider CCN'] != 440007]

In [55]:
#Targets dataset on Colorad0
#mainFile3 = mainFile3.loc[df['State Code'] == 'CO']

In [56]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'PR']

In [57]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'GU']

In [58]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'VI']

In [59]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'MP']

In [60]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'HI']

In [61]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'AK']

In [62]:
#Quick Check ... Lose 117 to state code reduction
mainFile3['Provider CCN'].count()

6144

In [63]:
#Remove hospitals that did not report revenue (Lose 273 to missing or unreported revenue events)
mainFile3 = mainFile3.loc[mainFile3['Net Patient Revenue'] >= 0.1]

In [64]:
#Quick Check
mainFile3['Provider CCN'].count()

5888

In [65]:
#Remove hospitals that discharged less than 15 patients (Lose 49 by placing a 15 patient minimum for study eligibility)
mainFile3 = mainFile3.loc[mainFile3['Total Discharges (V + XVIII + XIX + Unknown)'] >= 15]

In [66]:
#Quick Check
mainFile3['Provider CCN'].count()

5843

In [67]:
#Remove hospitals with less than 10 employees on payroll (Lose 23 by placing a 10 employee minimum for study eligibility)
mainFile3 = mainFile3.loc[mainFile3['FTE - Employees on Payroll'] >= 10]

In [68]:
#Quick Check
mainFile3['Provider CCN'].count()

5805

In [69]:
#Targets dataset on certificate of need states
#mainFile3 = mainFile3.loc[mainFile2['CON'] == 1]

In [70]:
#Remove net income outliers
#mainFile3 = mainFile3.loc[mainFile2['Net Income %'] <= .3304]

In [71]:
#Remove net income outliers
#mainFile3 = mainFile3.loc[mainFile2['Net Income %'] >= -.2473]

In [72]:
#Remove Discharges Per Bed outliers
#mainFile3 = mainFile3.loc[mainFile2['Rev Per Discharge'] <= 90000]

In [73]:
#Remove Rev Per Bed outliers
#mainFile3 = mainFile3.loc[mainFile2['Rev Per Bed'] <= 2000000]

In [74]:
#Remove Rev Per FTE outliers
#mainFile3 = mainFile3.loc[mainFile2['Rev Per FTE'] <= 500000]

In [75]:
#Remove Rev Per Discharge outliers
#mainFile3 = mainFile3.loc[mainFile2['Rev Per Discharge'] <= 175000]

In [76]:
#Targets dataset on STH locations (~56% of eligible continental US locations are STH providers)
#mainFile3 = mainFile3.loc[mainFile2['CCN Facility Type'] == 'STH']

In [77]:
#Quick Check
mainFile3['Provider CCN'].count()

5805

In [78]:
#Quick check on distribution of bronze, silver and gold
#mainFile3['Net Income Score'].value_counts()

In [79]:
#Quick Check
mainFile3['Provider CCN'].count()

5805

In [80]:
#mainFile3.to_csv("main3_2015.csv", index=False, header=True)

In [81]:
# ... Target file based on relevant features

In [82]:
mainFile4 = mainFile3

In [83]:
# #%'s'
# chg1percent = mainFile4["Net Income %"]
# chg1percent = chg1percent *100
# mainFile4["Net Income %"] = chg1percent
# mainFile4["Net Income %"] = mainFile4["Net Income %"].map("{:.4f}%".format)

# chg2percent = mainFile4["Labor %"]
# chg2percent = chg2percent *100
# mainFile4["Labor %"] = chg2percent
# mainFile4["Labor %"] = mainFile4["Labor %"].map("{:.4f}%".format)

# chg3percent = mainFile4["Overhead %"]
# chg3percent = chg3percent *100
# mainFile4["Overhead %"] = chg3percent
# mainFile4["Overhead %"] = mainFile4["Overhead %"].map("{:.4f}%".format)

# chg4percent = mainFile4["Depreciation %"]
# chg4percent = chg4percent *100
# mainFile4["Depreciation %"] = chg4percent
# mainFile4["Depreciation %"] = mainFile4["Depreciation %"].map("{:.4f}%".format)

# chg5percent = mainFile4["Charity %"]
# chg5percent = chg5percent *100
# mainFile4["Charity %"] = chg5percent
# mainFile4["Charity %"] = mainFile4["Charity %"].map("{:.4f}%".format)

# chg6percent = mainFile4["Uncompensated %"]
# chg6percent = chg6percent *100
# mainFile4["Uncompensated %"] = chg6percent
# mainFile4["Uncompensated %"] = mainFile4["Uncompensated %"].map("{:.4f}%".format)

# chg7percent = mainFile4["Inpatient %"]
# chg7percent = chg7percent *100
# mainFile4["Inpatient %"] = chg7percent
# mainFile4["Inpatient %"] = mainFile4["Inpatient %"].map("{:.4f}%".format)

# chg8percent = mainFile4["Outpatient %"]
# chg8percent = chg8percent *100
# mainFile4["Outpatient %"] = chg8percent
# mainFile4["Outpatient %"] = mainFile4["Outpatient %"].map("{:.4f}%".format)

# #$'s'
# mainFile4["Net Patient Revenue"] = mainFile4["Net Patient Revenue"].map("${:,.0f}".format)
# mainFile4["Net Income"] = mainFile4["Net Income"].map("${:.0f}".format)
# mainFile4["Wage-Related Costs (Core)"] = mainFile4["Wage-Related Costs (Core)"].map("${:,.0f}".format)
# mainFile4["Total Salaries (adjusted)"] = mainFile4["Total Salaries (adjusted)"].map("${:,.0f}".format)
# mainFile4["Contract Labor"] = mainFile4["Contract Labor"].map("${:,.0f}".format)
# mainFile4["Rev Per FTE"] = mainFile4["Rev Per FTE"].map("${:,.0f}".format)
# mainFile4["Rev/Discharge"] = mainFile4["Rev/Discharge"].map("${:,.0f}".format)
# mainFile4["Rev Per Bed"] = mainFile4["Rev Per Bed"].map("${:,.0f}".format)
# mainFile4["Rev Per Bed Days"] = mainFile4["Rev Per Bed Days"].map("${:,.0f}".format)
# mainFile4["LbrExp/FTE"] = mainFile4["LbrExp/FTE"].map("${:,.0f}".format)
# mainFile4["Overhead Exp Per Bed"] = mainFile4["Overhead Exp Per Bed"].map("${:,.0f}".format)

# ##'s'
# mainFile4["Discharges Per Bed"] = mainFile4["Discharges Per Bed"].map("{:,.2f}".format)
# mainFile4["Discharges Per Bed Days"] = mainFile4["Discharges Per Bed Days"].map("{:,.2f}".format)
# mainFile4["Discharge/FTE"] = mainFile4["Discharge/FTE"].map("{:,.2f}".format)

In [84]:
#Targeting the dataset on metrics important to Kent
mainFile4 = mainFile4[['Provider CCN','Hospital Name','Street Address','City','State Code','Zip Code', 'lat', 'lng', 'Rural Versus Urban', 'CCN Facility Type', 'Fiscal Year Begin Date', 'Fiscal Year End Date', 'FTE - Employees on Payroll', 'Number of Beds', 'Total Bed Days Available', 'Total Discharges (V + XVIII + XIX + Unknown)', 'Net Patient Revenue', 'Net Income', 'Wage-Related Costs (Core)', 'Total Salaries (adjusted)', 'Contract Labor', 'Rev Per FTE','CON','Urban','Inpatient %','Outpatient %','Charity %','Uncompensated %','Rev/Discharge','Rev Per Bed','Rev Per Bed Days', 'Discharges Per Bed','Discharges Per Bed Days','Discharge/FTE','Net Income %', 'Net Income Score', 'Labor %', 'Overhead %', 'Depreciation %', 'LbrExp/FTE', 'Overhead Exp Per Bed']]
mainFile4.head(2)


,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,lat,lng,Rural Versus Urban,CCN Facility Type,...,Discharges Per Bed,Discharges Per Bed Days,Discharge/FTE,Net Income %,Net Income Score,Labor %,Overhead %,Depreciation %,LbrExp/FTE,Overhead Exp Per Bed
1,110050,MURRAY MEDICAL CENTER,707 OLDS ELLIJAY HIGHWAY,CHATSWORTH,GA,30705,34.779956,-84.783038,U,STH,...,8.068966,0.038061,8.068966,-0.188530,3,0.660047,0.745925,0.138715,29311.008179,136861.793103
2,511312,PRESTON MEMORIAL HOSPITAL,150 MEMORIAL DRIVE,KINGWOOD,WV,26537-,39.465136,-79.677749,R,CAH,...,16.840000,0.093039,16.840000,0.097208,2,NaN,0.511725,0.025549,NaN,345736.720000


In [85]:
# Identify incomplete rows
mainFile4.count()

Provider CCN                                    5805
Hospital Name                                   5805
Street Address                                  5802
City                                            5805
State Code                                      5805
Zip Code                                        5805
lat                                             5639
lng                                             5639
Rural Versus Urban                              5805
CCN Facility Type                               5805
Fiscal Year Begin Date                          5805
Fiscal Year End Date                            5805
FTE - Employees on Payroll                      5805
Number of Beds                                  5805
Total Bed Days Available                        5805
Total Discharges (V + XVIII + XIX + Unknown)    5805
Net Patient Revenue                             5805
Net Income                                      5800
Wage-Related Costs (Core)                     

In [86]:
mainFile4 = mainFile4.dropna(how='any')

In [87]:
# Identify incomplete rows
mainFile4.count()

Provider CCN                                    2734
Hospital Name                                   2734
Street Address                                  2734
City                                            2734
State Code                                      2734
Zip Code                                        2734
lat                                             2734
lng                                             2734
Rural Versus Urban                              2734
CCN Facility Type                               2734
Fiscal Year Begin Date                          2734
Fiscal Year End Date                            2734
FTE - Employees on Payroll                      2734
Number of Beds                                  2734
Total Bed Days Available                        2734
Total Discharges (V + XVIII + XIX + Unknown)    2734
Net Patient Revenue                             2734
Net Income                                      2734
Wage-Related Costs (Core)                     

In [88]:
#%'s'
chg1percent = mainFile4["Net Income %"]
chg1percent = chg1percent *100
mainFile4["Net Income %"] = chg1percent
mainFile4["Net Income %"] = mainFile4["Net Income %"].map("{:.4f}%".format)

chg2percent = mainFile4["Labor %"]
chg2percent = chg2percent *100
mainFile4["Labor %"] = chg2percent
mainFile4["Labor %"] = mainFile4["Labor %"].map("{:.4f}%".format)

chg3percent = mainFile4["Overhead %"]
chg3percent = chg3percent *100
mainFile4["Overhead %"] = chg3percent
mainFile4["Overhead %"] = mainFile4["Overhead %"].map("{:.4f}%".format)

chg4percent = mainFile4["Depreciation %"]
chg4percent = chg4percent *100
mainFile4["Depreciation %"] = chg4percent
mainFile4["Depreciation %"] = mainFile4["Depreciation %"].map("{:.4f}%".format)

chg5percent = mainFile4["Charity %"]
chg5percent = chg5percent *100
mainFile4["Charity %"] = chg5percent
mainFile4["Charity %"] = mainFile4["Charity %"].map("{:.4f}%".format)

chg6percent = mainFile4["Uncompensated %"]
chg6percent = chg6percent *100
mainFile4["Uncompensated %"] = chg6percent
mainFile4["Uncompensated %"] = mainFile4["Uncompensated %"].map("{:.4f}%".format)

chg7percent = mainFile4["Inpatient %"]
chg7percent = chg7percent *100
mainFile4["Inpatient %"] = chg7percent
mainFile4["Inpatient %"] = mainFile4["Inpatient %"].map("{:.4f}%".format)

chg8percent = mainFile4["Outpatient %"]
chg8percent = chg8percent *100
mainFile4["Outpatient %"] = chg8percent
mainFile4["Outpatient %"] = mainFile4["Outpatient %"].map("{:.4f}%".format)

#$'s'
mainFile4["Net Patient Revenue"] = mainFile4["Net Patient Revenue"].map("${:,.0f}".format)
mainFile4["Net Income"] = mainFile4["Net Income"].map("${:.0f}".format)
mainFile4["Wage-Related Costs (Core)"] = mainFile4["Wage-Related Costs (Core)"].map("${:,.0f}".format)
mainFile4["Total Salaries (adjusted)"] = mainFile4["Total Salaries (adjusted)"].map("${:,.0f}".format)
mainFile4["Contract Labor"] = mainFile4["Contract Labor"].map("${:,.0f}".format)
mainFile4["Rev Per FTE"] = mainFile4["Rev Per FTE"].map("${:,.0f}".format)
mainFile4["Rev/Discharge"] = mainFile4["Rev/Discharge"].map("${:,.0f}".format)
mainFile4["Rev Per Bed"] = mainFile4["Rev Per Bed"].map("${:,.0f}".format)
mainFile4["Rev Per Bed Days"] = mainFile4["Rev Per Bed Days"].map("${:,.0f}".format)
mainFile4["LbrExp/FTE"] = mainFile4["LbrExp/FTE"].map("${:,.0f}".format)
mainFile4["Overhead Exp Per Bed"] = mainFile4["Overhead Exp Per Bed"].map("${:,.0f}".format)

##'s'
mainFile4["Discharges Per Bed"] = mainFile4["Discharges Per Bed"].map("{:,.2f}".format)
mainFile4["Discharges Per Bed Days"] = mainFile4["Discharges Per Bed Days"].map("{:,.2f}".format)
mainFile4["Discharge/FTE"] = mainFile4["Discharge/FTE"].map("{:,.2f}".format)

In [89]:
#Output targeted file
mainFile4.to_csv("main4_2015.csv", index=False, header=True)

In [85]:
# ... Net Income to feature review

In [86]:
targetFile5 = mainFile4

In [87]:
#Outlier Review
chartdata = targetFile5["Discharges Per Bed"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of profit percent is: {lowerq}")
print(f"The upper quartile of profit percent is: {upperq}")
print(f"The interquartile range of profit percent is: {iqr}")
print(f"The the median of profit percent is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile of profit percent is: 32.692739079102715
The upper quartile of profit percent is: 54.410707383069195
The interquartile range of profit percent is: 21.71796830396648
The the median of profit percent is: 44.94809417040359 
Values below 0.11578662315299226 could be outliers.
Values above 86.98765983901892 could be outliers.


In [88]:
#NI v. discharges per bed
# x_axis = targetFile5["Discharges Per Bed"]
# y_axis = targetFile5["Net Income %"]
# plt.title("Net Income % v. Discharges Per Bed")
# plt.xlabel("Discharges Per Bed")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [89]:
#NI v. discharges per bed days
# x_axis = targetFile5["Discharges Per Bed Days"]
# y_axis = targetFile5["Net Income %"]
# plt.title("Net Income % v. Discharges Per Bed Days")
# plt.xlabel("Discharges Per Bed Days")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# #ax.set_yscale('log')
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [90]:
#Outlier Review
chartdata = targetFile5["Rev Per Bed"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of profit percent is: {lowerq}")
print(f"The upper quartile of profit percent is: {upperq}")
print(f"The interquartile range of profit percent is: {iqr}")
print(f"The the median of profit percent is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile of profit percent is: 757189.0897272727
The upper quartile of profit percent is: 1480425.1838256354
The interquartile range of profit percent is: 723236.0940983626
The the median of profit percent is: 1075917.8136524823 
Values below -327665.05142027116 could be outliers.
Values above 2565279.324973179 could be outliers.


In [91]:
#NI v. Revenue Per Bed
# x_axis = targetFile5["Rev Per Bed"]
# y_axis = targetFile5["Net Income %"]
# plt.title("Net Income % v. Rev Per Bed")
# plt.xlabel("Rev Per Bed")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [93]:
#Outlier Review
# chartdata = targetFile5["Rev/FTE"]
# quartiles = chartdata.quantile([.25,.5,.75])
# lowerq = quartiles[0.25]
# upperq = quartiles[0.75]
# iqr = upperq-lowerq

# print(f"The lower quartile is: {lowerq}")
# print(f"The upper quartile is: {upperq}")
# print(f"The interquartile range is: {iqr}")
# print(f"The the median is: {quartiles[0.5]} ")

# lower_bound = lowerq - (1.5*iqr)
# upper_bound = upperq + (1.5*iqr)
# print(f"Values below {lower_bound} could be outliers.")
# print(f"Values above {upper_bound} could be outliers.")

In [94]:
#NI v. Rev Per FTE
targetFile6 = targetFile5
# targetFile6 = targetFile5.loc[targetFile5['Rev Per FTE'] <= 1000000]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Rev Per FTE"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Rev Per FTE")
# plt.xlabel("Rev Per FTE")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [95]:
#Outlier Review
chartdata = targetFile5["Rev/Discharge"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of profit percent is: {lowerq}")
print(f"The upper quartile of profit percent is: {upperq}")
print(f"The interquartile range of profit percent is: {iqr}")
print(f"The the median of profit percent is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile of profit percent is: 19401.29180066132
The upper quartile of profit percent is: 32498.537874122925
The interquartile range of profit percent is: 13097.246073461603
The the median of profit percent is: 24170.934149524634 
Values below -244.5773095310833 could be outliers.
Values above 52144.40698431533 could be outliers.


In [96]:
#NI v. Rev Per Discharge
# x_axis = targetFile5["Rev Per Discharge"]
# y_axis = targetFile5["Net Income %"]
# plt.title("Net Income % v. Rev Per Discharge")
# plt.xlabel("Rev Per Discharge")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [97]:
#Outlier Review
chartdata = targetFile5["LbrExp/FTE"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile is: {lowerq}")
print(f"The upper quartile is: {upperq}")
print(f"The interquartile range is: {iqr}")
print(f"The the median of is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile is: 71965.48681927467
The upper quartile is: 92410.90329649393
The interquartile range is: 20445.41647721926
The the median of is: 81399.06628316542 
Values below 41297.362103445776 could be outliers.
Values above 123079.02801232281 could be outliers.


In [98]:
#NI v. Labor Expense Per FTE
# targetFile6 = targetFile5.loc[targetFile5['Labor Exp Per FTE'] <= 130000]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Labor Exp Per FTE"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Labor Exp Per FTE")
# plt.xlabel("Labor Exp Per FTE")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [99]:
#Outlier Review
chartdata = targetFile5["Overhead %"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile is: {lowerq}")
print(f"The upper quartile is: {upperq}")
print(f"The interquartile range is: {iqr}")
print(f"The the median of is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile is: 0.5545099840076749
The upper quartile is: 0.6772157393237679
The interquartile range is: 0.12270575531609296
The the median of is: 0.6097646044426033 
Values below 0.3704513510335355 could be outliers.
Values above 0.8612743722979073 could be outliers.


In [100]:
#NI v. Overhead Percent
# targetFile6 = targetFile5.loc[targetFile5['Overhead %'] <= .90]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Overhead %"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Overhead %")
# plt.xlabel("Overhead %")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [101]:
#Outlier Review
chartdata = targetFile5["Labor %"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile is: {lowerq}")
print(f"The upper quartile is: {upperq}")
print(f"The interquartile range is: {iqr}")
print(f"The the median of is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile is: 0.3763138393208173
The upper quartile is: 0.5306868932538241
The interquartile range is: 0.15437305393300682
The the median of is: 0.4493513642420821 
Values below 0.14475425842130707 could be outliers.
Values above 0.7622464741533344 could be outliers.


In [102]:
#NI v. Labor Percent
targetFile6 = targetFile5
# targetFile6 = targetFile5.loc[targetFile5['Labor %'] <= .90]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Labor %"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Labor %")
# plt.xlabel("Labor %")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [103]:
#Outlier Review
chartdata = targetFile5["Discharge/FTE"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile is: {lowerq}")
print(f"The upper quartile is: {upperq}")
print(f"The interquartile range is: {iqr}")
print(f"The the median of is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile is: 32.692739079102715
The upper quartile is: 54.410707383069195
The interquartile range is: 21.71796830396648
The the median of is: 44.94809417040359 
Values below 0.11578662315299226 could be outliers.
Values above 86.98765983901892 could be outliers.


In [104]:
#NI v. Labor Percent
# targetFile6 = targetFile5
# targetFile6 = targetFile5.loc[targetFile5['Discharges Per FTE'] <= 120]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Discharges Per FTE"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Discharges Per FTE")
# plt.xlabel("Discharges Per FTE")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [105]:
#Start machine learning ...

In [106]:
# Select features to be used as x values
features1 = mainFile4[['CON','Urban','Inpatient %','Outpatient %','Charity %','Uncompensated %','Rev/Discharge','Rev Per Bed','Rev Per Bed Days', 'Discharges Per Bed','Discharges Per Bed Days','Discharge/FTE', 'Labor %', 'Overhead %', 'Depreciation %', 'LbrExp/FTE', 'Overhead Exp Per Bed']]
features1.head(2)
#RFE logrithmic output ... array([15, 17, 16, 14, 12,  8, 11,  6,  5 (R/BDay),  7,  9,  3(D/FTE),  2(L%),  1 (OH%),  4 (D%), 13, 10])

,CON,Urban,Inpatient %,Outpatient %,Charity %,Uncompensated %,Rev/Discharge,Rev Per Bed,Rev Per Bed Days,Discharges Per Bed,Discharges Per Bed Days,Discharge/FTE,Labor %,Overhead %,Depreciation %,LbrExp/FTE,Overhead Exp Per Bed
1,1.0,1.0,0.116988,0.883012,0.003257,0.003257,22738.897436,183479.37931,865.468770,8.068966,0.038061,8.068966,0.660047,0.745925,0.138715,29311.008179,136861.793103
54,1.0,0.0,0.172685,0.827315,0.012981,0.012981,23937.533333,718126.00000,1967.468493,30.000000,0.082192,30.000000,0.890036,0.746006,0.169899,52809.873585,535726.000000


In [107]:
#Output features file as a reference document
features1.to_csv("features2015.csv", index=False, header=True)

In [108]:
# ... Linear Regression Model

In [109]:
# Identify the outcome column/data
y = mainFile4[['Net Income %']]
y.head()

,Net Income %
1,-0.188530
54,-0.161392
79,-0.121009
142,-0.700503
223,0.306294


In [110]:
mainFile4["Net Income %"].dtype

dtype('float64')

In [111]:
y.shape

(3060, 1)

In [112]:
X = features1
X.shape

(3060, 17)

In [113]:
#Create a training and testing dataset - one hot
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [114]:
y_train.shape

(2295, 1)

In [115]:
X_train.shape

(2295, 17)

In [116]:
y_test.shape

(765, 1)

In [117]:
X_test.shape

(765, 17)

In [118]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [119]:
#Create the linear regression model and fit to the training data
#score is r2 root mean ... 1 is better
modela = LinearRegression()
modela.fit(X_train_scaled, y_train)
score = modela.score(X_train_scaled, y_train)
print(f"Training Data Score: {modela.score(X_train_scaled, y_train)}")

Training Data Score: 0.5607933670893857


In [120]:
#Run the model with the testing data and determine score
print(f"Testing Data Score: {modela.score(X_test_scaled, y_test)}")

Testing Data Score: -0.7044069124716761


In [121]:
# ... Linear Recursive Feature Elimination

In [122]:
#Identify features to create a more efficient model with
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=5, step=1)
selector = selector.fit(X_train_scaled, y_train)

In [123]:
selector.support_

array([False, False, False, False, False, False, False,  True, False,
       False, False, False,  True,  True,  True, False,  True])

In [124]:
selector.ranking_

array([10, 13,  7, 12,  5,  4,  3,  1,  6, 11,  8,  9,  1,  1,  1,  2,  1])

In [125]:
# ... Logistics Model

In [126]:
# Identify the outcome column/data
yy = mainFile4[['Net Income Score']]
yy.head()

,Net Income Score
1,3
54,3
79,3
142,3
223,1


In [127]:
yy.shape

(3060, 1)

In [128]:
XX = features1
XX.shape

(3060, 17)

In [129]:
#Create a training and testing dataset - one hot
X_train, X_test, y_train, y_test = train_test_split(
    XX, yy, random_state=1)

In [130]:
y_train.shape

(2295, 1)

In [131]:
X_train.shape

(2295, 17)

In [132]:
y_test.shape

(765, 1)

In [133]:
y_test.shape

(765, 1)

In [134]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [135]:
#Create the logistic regression model and fit to the training data
#score is r2 root mean ... 1 is better
modela = LogisticRegression()
modela.fit(X_train_scaled, y_train)
score = modela.score(X_train_scaled, y_train)
print(f"Training Data Score: {modela.score(X_train_scaled, y_train)}")

Training Data Score: 0.6026143790849673


C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [136]:
#Run the model with the testing data and determine score
print(f"Testing Data Score: {modela.score(X_test_scaled, y_test)}")

Testing Data Score: 0.6052287581699346


In [137]:
# ... recursive feature elimination

In [138]:
#Identify features to create a more efficient model with
estimator = LogisticRegression()
selector = RFE(estimator, n_features_to_select=1, step=1)
selector = selector.fit(X_train_scaled, y_train)

C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pleas

In [139]:
selector.support_

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False])

In [140]:
selector.ranking_

array([13, 16, 17, 14, 12,  9, 10,  3,  5,  8, 15,  7,  2,  1,  4, 11,  6])

In [ ]:
#End ... kent matthew rodgers, profitableHospital-project